In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import time
from sklearn.metrics import accuracy_score,mean_squared_error,mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import tree
import graphviz
import shap
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('Auction_and_demand_data_MAIN.csv')

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
sns.jointplot(x='NSL_FLOW',y='Average base prices',data=data,kind='reg')

#  ploting the flow of data in time

In [ ]:
data['Date'] = pd.to_datetime(data['Date'], utc=True, infer_datetime_format=True)
data = data.set_index('Date')

In [ ]:
def plot_series(df=None, column=None, series=pd.Series([]), 
                label=None, ylabel=None, title=None, start=0, end=None):
    sns.set()
    fig, ax = plt.subplots(figsize=(30, 12))
    ax.set_xlabel('Time', fontsize=16)
    if column:
        ax.plot(df[column][start:end], label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if series.any():
        ax.plot(series, label=label)
        ax.set_ylabel(ylabel, fontsize=16)
    if label:
        ax.legend(fontsize=16)
    if title:
        ax.set_title(title, fontsize=24)
    ax.grid(True)
    return ax

In [ ]:
ax = plot_series(df=data, column='3_Hour', ylabel='Average base prices',
                  title='cost of price per hour')
plt.show()

# Checking for correlatin bewteen the target(Average base price ) and other features

In [ ]:
data.corr()
data.corr().iloc[:,24:]

In [ ]:
data = data.dropna(axis=0)

In [ ]:
data.columns

In [ ]:
data['NSL_FLOW'].value_counts()

In [ ]:
corr_matrix = data.corr().abs()
corr_matrix

In [ ]:
data['Average base prices'].isna().sum()

In [ ]:
corr1 = data.corr('pearson')[['Average base prices']].sort_values(by='Average base prices', ascending=False)
(corr1 > 0.70).sum()

# Manually selecting the features for prediction

In [ ]:
X = data.drop(columns=['PUMP_STORAGE_PUMPING','IFA_FLOW','IFA2_FLOW','BRITNED_FLOW','MOYLE_FLOW','EAST_WEST_FLOW',
                      'NEMO_FLOW','ELECLINK_FLOW','Average base prices','Week','Month','ND','TSD','ENGLAND_WALES_DEMAND',
                       'EMBEDDED_WIND_GENERATION','EMBEDDED_WIND_CAPACITY','EMBEDDED_SOLAR_GENERATION','EMBEDDED_SOLAR_CAPACITY','NON_BM_STOR'])
y = data['Average base prices']

In [ ]:
d = data[['1_Hour', '2_Hour', '3_Hour', '4_Hour', '5_Hour', '6_Hour', '7_Hour','8_Hour', '9_Hour', '10_Hour', '11_Hour', '12_Hour', '13_Hour', '14_Hour', '15_Hour', '16_Hour', '17_Hour', '18_Hour', '19_Hour', '20_Hour', '21_Hour', '22_Hour', '23_Hour', '24_Hour', 'NSL_FLOW','Average base prices']]

In [ ]:
fig = plt.figure(figsize=(30,20))
sns.heatmap(d.corr(),cmap='icefire_r',linewidths=0.30,annot=True,)
fig.savefig("Correlation of features", bbox_inches='tight', dpi=600)

In [ ]:
X.shape

In [ ]:
X.describe()

# checking the dimension of the data with plot

In [ ]:
data.plot(kind='line', x='1_Hour', y='Average base prices');
data.plot(kind='line', x='NSL_FLOW', y='Average base prices');

In [ ]:
sns.boxplot(x= data['2_Hour'])

In [ ]:
X.info()

In [ ]:
# This will take time to plot beacuse it will plot each feature against others as to check for relationship betwwen that feature and others in the dataset


# sns.pairplot(X, markers="+", kind='reg',
#              diag_kind="auto",
#              plot_kws={'line_kws':{'color':'#aec6cf'},
#                        'scatter_kws': {'alpha': 0.5,
#                                        'color': '#82ad32'}},
#              diag_kws= {'color': '#82ad32'})


# standardizing the data in other to bring all the unit to the same range

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X)

# Spliting the data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [ ]:
print(f'x_train: {x_train.shape}, {y_train.shape}')
print(f'x_train: {x_test.shape}, {y_test.shape}')

# Using RandomForest to train the model 

In [ ]:
model_1 = RandomForestRegressor(random_state =8)

# Fitting the model

In [ ]:
model_1.fit(x_train,y_train)

In [ ]:
model_1.score(x_train,y_train)

In [ ]:
model_1.score(x_test,y_test)


In [ ]:
fig = plt.figure(figsize=(20,10))
feat_importances = pd.DataFrame(model_1.feature_importances_, index=x_train.columns, columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
feat_importances.plot(kind='bar', figsize=(8,6),title='Random Forest Feature Importance')
fig.savefig("Random Forest Feature Importances.png", bbox_inches='tight', dpi=600,)

In [ ]:
y_pred= model_1.predict(x_test)
y_pred

In [ ]:
mean_squared_error(y_test,y_pred,squared=False)

In [ ]:
print ('Random Forest')
print("MAE:",mean_absolute_error(y_test,y_pred))
print ("MSE:",mean_squared_error(y_test,y_pred))
print("RMSE:",np.sqrt(mean_squared_error(y_test,y_pred)))

# Using Feature selection to get us the best features to use for prediction

In [ ]:
sel = SelectKBest(k=12)

In [ ]:
sel.fit(X,y)

In [ ]:
sel.get_support()

In [ ]:
X.columns[sel.get_support()]

In [ ]:
new_x = data[['4_Hour', '7_Hour', '14_Hour','15_Hour', '16_Hour', '17_Hour', '18_Hour',
       '19_Hour','20_Hour', '21_Hour', '22_Hour','NSL_FLOW']]

In [ ]:
new_x.head()

In [ ]:
new_x_train,new_x_test,new_y_train,new_y_test = train_test_split(new_x,y,test_size=0.2,random_state=2)

# Buiding Decision Tree Model for prediction

In [ ]:
model_2 = DecisionTreeRegressor(random_state=4, max_depth=6)

In [ ]:
model_2.fit(new_x_train,new_y_train)

In [ ]:
new_y_pred= model_2.predict(new_x_test)
new_y_pred

In [ ]:
print (mean_squared_error(y_test,new_y_pred,squared=False))


In [ ]:
fig = plt.figure(figsize=(20,10))
feat_importances = pd.DataFrame(model_2.feature_importances_, index=new_x_train.columns, columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
feat_importances.plot(kind='barh', figsize=(8,6),color='red', title='DecisionTree Feature Importance')
fig.savefig("DecisionTree Feature Importances.png", bbox_inches='tight', dpi=600)

In [ ]:
print ('Decision Tree')
print ('Test score', model_2.score(new_x_train,new_y_train))
print ('Train score', model_2.score(new_x_test,new_y_test))

In [ ]:
print ('Decision Tree')
print("MAE:",mean_absolute_error(y_test,new_y_pred))
print ("MSE:",mean_squared_error(y_test,new_y_pred))
print("RMSE:",np.sqrt(mean_squared_error(y_test,new_y_pred)))

# Using Support Vector Regression

In [ ]:
model_3 = SVR()
model_3.fit(x_train,y_train)

In [ ]:
model_3.score(x_train,y_train)

In [ ]:
model_3.score(x_test,y_test)

In [ ]:
from sklearn.inspection import permutation_importance
import matplotlib. pyplot as pyplot
results = permutation_importance(model_3, x_train, y_train, scoring='neg_mean_squared_error')
# get importance
importance = results.importances_mean
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
#plot feature importance
fig = plt.figure(figsize=(20,10))
plt.title('SVR Feature Permutation Importance')
pyplot.bar([x for x in range(len(importance))], importance)
pyplot.show()
fig.savefig("SVR Feature Permutation Importance .png", bbox_inches='tight', dpi=600)

In [ ]:
y_pred_3= model_3.predict(x_test)
y_pred_3

In [ ]:
mean_squared_error(y_test,y_pred_3,squared=False)

In [ ]:
print ('SVR')
print("MAE:",mean_absolute_error(y_test,y_pred_3))
print ("MSE:",mean_squared_error(y_test,y_pred_3))
print("RMSE:",np.sqrt(mean_squared_error(y_test,y_pred_3)))

# Explaining the model with shap model on Randomforest 

In [ ]:
explainer_model_1 = shap.TreeExplainer(model_1)

In [ ]:
base_value_1 = explainer_model_1.expected_value
base_value_1

In [ ]:
shap_value_1 = explainer_model_1.shap_values(x_test)

In [ ]:
figure = plt.figure()
shap.summary_plot(shap_value_1, x_test)
figure.savefig("summary_plot1.png", bbox_inches='tight', dpi=600)

In [ ]:
shap.dependence_plot("19_Hour", shap_value_1, x_test)

In [ ]:
shap.force_plot(explainer_model_1.expected_value, shap_value_1[99,:], x_test.iloc[99,:],matplotlib=True)

In [ ]:
print(model_1.feature_importances_)
importances = model_1.feature_importances_
indices = np.argsort(importances)
features = x_train.columns
plt.figure(figsize=(30,20))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


In [ ]:
y_test.mean()

# Explaining the Decision Tress model  

In [ ]:
fig = plt.figure(figsize=(50,40))
_ = tree.plot_tree(model_2, feature_names=X.columns, class_names=y.unique, filled=True, 
                   fontsize=16)


# Explaining SVR model with shap model 

In [ ]:
explainer_model_3 = shap.KernelExplainer(model_3.predict,x_test)

In [ ]:
base_value_3 = explainer_model_3.expected_value
base_value_3

In [ ]:
shap_value_3 = explainer_model_3.shap_values(x_test)

In [ ]:
shap.summary_plot(shap_value_3, x_test)

# Building of sequential layer 

In [ ]:
ann = Sequential()                          
ann.add(Dense(50,input_shape=(25,), activation='relu'))  
ann.add(Dense(units=10, activation="relu"))  
ann.add(Dense(units=5, activation='tanh'))
ann.add(Dense(units=25))   
ann.compile(optimizer="adam",loss="mean_squared_error")

In [ ]:
ann.fit(x=x_train, y=y_train, epochs=15, batch_size=5,validation_data=(x_test,y_test))

In [ ]:
plt.style.use("ggplot")
pd.DataFrame(ann.history.history).plot(figsize=(12,10))
print(pd.DataFrame(ann.history.history))

In [ ]:
ann.evaluate(x_train,y_train)

In [ ]:
ann.evaluate(x_test,y_test)

In [ ]:
predictions = ann.predict(x_test).sum(axis=1)

In [ ]:
print(ann.history.history)

# metrics to see how good the model perform 
the lower the metric the better the prediction

In [ ]:
print("MAE:",mean_absolute_error(y_test,predictions))
print ("MSE:",mean_squared_error(y_test,predictions))
print("RMSE:",np.sqrt(mean_squared_error(y_test,predictions)))

# Ploting of graph between Actual Values and Predictions on ANN Model

In [ ]:
plt.figure(figsize=(12,10))
sns.scatterplot(np.ravel(predictions),y_test)
plt.title("The Scatterplot of Relationship between Actual Values and Predictions")
plt.xlabel("Predictions")
plt.ylabel("Actual Values")

In [ ]:

for more understanding on explainable AI you can check out the link
# https://medium.com/dataman-in-ai/explain-your-model-with-the-shap-values-bc36aac4de3d